<a href="https://colab.research.google.com/github/BuiltByG/UWI2025CRSComplianceRAGTeam/blob/UWICRSComplianceRAG_GoogleColab/crsredo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Initial cell

In [ ]:
# Downgrade NumPy to fix FAISS compatibility issues
!pip install "numpy<2.0.0"

# Force kernel restart after install
import os
import IPython
os.kill(os.getpid(), 9)


Rerun

In [1]:
import os

In [2]:
os.environ["NVIDIA_API_KEY"] = ">>INSERT API KEY HERE<<"

In [4]:
!pip install -q langchain langchain-nvidia-ai-endpoints gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00


In [5]:
!pip install -q langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00


In [6]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
ChatNVIDIA.get_available_models()

[Model(id='mediatek/breeze-7b-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=['ai-breeze-7b-instruct'], supports_tools=False, supports_structured_output=False, supports_thinking=False, base_model=None),
 Model(id='google/gemma-3-4b-it', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=False, supports_structured_output=False, supports_thinking=False, base_model=None),
 Model(id='google/gemma-2b', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=['ai-gemma-2b', 'playground_gemma_2b', 'gemma_2b'], supports_tools=False, supports_structured_output=False, supports_thinking=False, base_model=None),
 Model(id='deepseek-ai/deepseek-coder-6.7b-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=['ai-deepseek-coder-6_7b-instruct'], supports_tools=False, supports_structured_output=False, supports_thinking=False, base_model=None),
 Model(id='meta/codellama-70b', model_type='chat', client='ChatNVIDIA', 

In [23]:
from pathlib import Path
from pprint import pprint
import json

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings


In [8]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 9.6 MB/s eta 0:00:00


In [9]:
from google.colab import files
uploaded = files.upload()

Saving 841e9512-en.pdf to 841e9512-en.pdf


PDF

In [10]:
# Set your PDF directory
pdf_dir = Path(".")  # or specify a full path

# Load all PDF files in the directory
pdf_files = list(Path(".").glob("*.pdf"))
docs = [PyPDFLoader(str(pdf)).load() for pdf in pdf_files]

# Trim "References" section if present
for doc in docs:
    content = doc[0].page_content
    if "References" in content:
        doc[0].page_content = content[:content.index("References")]


Chunking (no soya)

In [11]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", ";", ",", " "],
)

# Split and flatten chunks
docs_chunks = []
for doc in docs:
    chunks = text_splitter.split_documents(doc)
    docs_chunks.extend(chunks)

# Optional: print basic stats
print(f"Total Chunks: {len(docs_chunks)}")
print("Example chunk:")
print(docs_chunks[0].page_content[:300])


Total Chunks: 466
Example chunk:
© OECD 2017 1
Standard for Automatic Exchange of 
Financial Information in Tax Matters
IMPLEMENTATION  
HANDBOOK
Second edition


In [12]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 69.8 MB/s eta 0:00:00


FAISS Vector

In [17]:
# Embed using NVIDIA’s cloud embedding endpoint
embedding_model = NVIDIAEmbeddings(model="nvidia/nv-embed-v1")

# Create or update the vector store
db = FAISS.from_documents(docs_chunks, embedding_model)

# Save to disk (optional, for reuse)
db.save_local("faiss_crs_index")


RAG Chain

In [21]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import UnstructuredPDFLoader

# Load vector store
print("Loading FAISS vector store")
db = FAISS.load_local("faiss_crs_index", NVIDIAEmbeddings(model="nvidia/nv-embed-v1"), allow_dangerous_deserialization=True)
retriever = db.as_retriever(search_kwargs={"k": 5})

# Define Mixtral model
llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")

# Create a simple prompt template
prompt_template = """Use the following context to answer the question.
If the answer cannot be found in the context, say you don't know.

Context:
{context}

Question:
{question}

Answer:"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# Build RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)


Loading FAISS vector store


Testing

In [22]:
query = "What are the rules for reporting on closed accounts?"
response = qa_chain.invoke({"query": query})

print("\nCRS Compliance Assistant:\n")
print(response["result"])



CRS Compliance Assistant:

According to the provided context, the rules for reporting on closed accounts are as follows:

1. Reporting the closure of the account: Under the Standard, only the fact that the account has been closed needs to be reported. This is considered a simplified approach for the Standard. Under the Model 1 FATCA IGA, Financial Institutions must report the account balance immediately before closure.

2. Reporting the account balance or value: Under the Standard, it is not required to report the account balance or value upon closure. However, under the Model 1 FATCA IGA, the account balance or value upon closure is still required.

3. Additional information: In case of closure of an account maintained by a trust that is a Reporting Financial Institution, or maintained by a Reporting Financial Institution for a trust that is a Passive NFE, the financial activity to be reported includes both the fact of closure of the account and the gross payments made to the Account

Gradio:

In [ ]:
import gradio as gr
import time

def stream_echo(message, history):
    buffer = ""
    for c in message:
        buffer += c
        time.sleep(0.1)
        yield history + [[message, buffer]]

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Say something...")
    state = gr.State([])

    msg.submit(stream_echo, inputs=[msg, state], outputs=[chatbot, state])

demo.launch(debug=True, share=True)
